# Práctica de Laboratorio: Introducción a RabbitMQ
### Arquitectura y Sistemas Distribuidos

---

## 1. Introducción Teórica

**RabbitMQ** es un intermediario de mensajes (Message Broker) de código abierto que implementa el protocolo **AMQP** (Advanced Message Queuing Protocol). Su función principal es actuar como un "buzón de correo" inteligente: acepta, almacena y reenvía mensajes entre aplicaciones.

### Componentes Clave:

1.  **Productores (Producers):** Aplicaciones que envían mensajes.
2.  **Consumidores (Consumers):** Aplicaciones que esperan recibir y procesar mensajes.
3.  **Colas (Queues):** El buzón dentro de RabbitMQ donde se almacenan los mensajes hasta que son consumidos. Una cola está limitada únicamente por los recursos del servidor (disco y memoria).
4.  **Exchanges:** Los productores no envían mensajes directamente a las colas. En su lugar, los envían a un *Exchange*, que decide a qué cola enviarlos según ciertas reglas (Bindings).

### El Patrón Productor-Consumidor
Este patrón permite el **desacoplamiento** de sistemas. El productor no necesita saber quién procesará el mensaje, ni si el consumidor está activo en ese momento. RabbitMQ garantiza que el mensaje se guarde de forma segura hasta que alguien lo pida.

## 2. Preparación del Entorno

Para esta práctica utilizaremos la librería `pika`, que es el cliente de Python oficial recomendado para RabbitMQ.

### Requisitos:
1.  **RabbitMQ Server:** Debe estar instalado y ejecutándose en `localhost` (o en un contenedor Docker).
2.  **Librería Pika:** Instalación mediante pip.

In [1]:
# Instalación de la librería necesaria (ejecutar si no se tiene instalada)
!pip install pika


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## 3. Implementación del Productor (Producer)

El productor se encarga de conectar con RabbitMQ, declarar una cola (para asegurar que existe) y enviar un mensaje.

In [2]:
import pika

# 1. Establecer conexión con el servidor local
connection = pika.BlockingConnection(
    pika.ConnectionParameters('localhost')
)
channel = connection.channel()

# 2. Declarar la cola (si no existe, se crea)
# Es una buena práctica declararla en ambos lados (productor y consumidor)
channel.queue_declare(queue='cola_prueba')

# 3. Enviar el mensaje
mensaje = "Hola desde el Notebook de RabbitMQ"
channel.basic_publish(
    exchange='', # Exchange por defecto
    routing_key='cola_prueba',
    body=mensaje
)

print(f" [x] Mensaje enviado: '{mensaje}'")

# 4. Cerrar la conexión
connection.close()

 [x] Mensaje enviado: 'Hola desde el Notebook de RabbitMQ'


## 4. Implementación del Consumidor (Consumer)

El consumidor se queda "escuchando" la cola. Cuando llega un mensaje, ejecuta una función de retorno (callback) para procesarlo.

In [3]:
import pika
import sys

def main():
    # 1. Conexión
    connection = pika.BlockingConnection(
        pika.ConnectionParameters('localhost')
    )
    channel = connection.channel()

    # 2. Declarar cola
    channel.queue_declare(queue='cola_prueba')

    # 3. Definir función de procesamiento (callback)
    def callback(ch, method, properties, body):
        print(f" [x] Recibido: {body.decode()}")

    # 4. Indicar que queremos consumir de la cola
    channel.basic_consume(
        queue='cola_prueba',
        on_message_callback=callback,
        auto_ack=True
    )

    print(' [*] Esperando mensajes. Para salir presiona CTRL+C')
    
    # 5. Iniciar el bucle de escucha
    try:
        channel.start_consuming()
    except KeyboardInterrupt:
        print('Interrumpido por el usuario')
        connection.close()

if __name__ == '__main__':
    # NOTA: En un Notebook, start_consuming() bloqueará la celda.
    # Se recomienda ejecutar el productor desde otra celda o script para ver el efecto.
    main()

 [*] Esperando mensajes. Para salir presiona CTRL+C
 [x] Recibido: Hola desde el Notebook de RabbitMQ
Interrumpido por el usuario


## 5. Desafíos para el Estudiante

Para profundizar en el aprendizaje, realice los siguientes ejercicios:

1.  **Modificar el Mensaje:** Cambie el contenido del mensaje enviado por el productor para incluir su nombre y la fecha actual.
2.  **Múltiples Mensajes:** Modifique el productor para que envíe 10 mensajes seguidos usando un bucle `for`.
3.  **Ack Manual:** Investigue para qué sirve `auto_ack=False` y cómo cambia la fiabilidad del sistema si el consumidor falla a mitad del procesamiento.
4.  **Dos Consumidores:** ¿Qué ocurre si ejecutas dos instancias del consumidor al mismo tiempo y envías varios mensajes? (Observe el reparto de carga o *Round Robin*).